In [39]:
import pandas as pd
from collections import Counter
import os
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
import numpy as np
from scipy import stats


data_dir = f'data'
project = 'battery-life-exp5'
dir = f'{data_dir}/{project}'

df = pd.DataFrame()
for file in os.listdir(dir):
    if file.startswith('Session'):
        with open(f'{dir}/{file}') as f:
            starttime_millis = int(f.readline().rstrip()[17:])
        session = pd.read_csv(f'{dir}/{file}', skiprows=1)
    else:
        tmp = pd.read_csv(f'{dir}/{file}', skiprows=1)
        df = pd.concat([df, tmp], ignore_index=True)
df = df.sort_values(by=['timestamp'], ignore_index=True)

df = df.loc[:len(df)/2, :]


print(len(df))
# get readable time (delta from start time)
real_times = []
for time_millis in df['real time']:
    delta = (time_millis - starttime_millis) # convert to seconds
    real_times.append(str(timedelta(milliseconds=delta)))
df['human time'] = real_times

# get time delta at each sample
deltas = []
for i, timestamp in enumerate(df['timestamp']):
    if i == 0:
        deltas.append(np.nan)
        continue
    # get time since last sample, convert to Hz
    deltas.append( (df['timestamp'][i] - df['timestamp'][i-1]) / 1e9)
df['delta'] = deltas

df.to_csv(f'{data_dir}/processed/{project}-processed.csv')

print(df['delta'].describe())
print(df)
import plotly.express as px

fig = px.line(df, x='human time', y='delta', title='Time delta', 
                labels={'human time': 'Time', 
                        'delta': 'Time delta (s)'})
fig.show(renderer='browser')


179173
count    179172.000000
mean          0.396411
std           9.468782
min           0.000003
25%           0.220000
50%           0.220000
75%           0.220000
max         513.680000
Name: delta, dtype: float64
              timestamp     acc_x     acc_y     acc_z      real time activity  \
0       187379899891333 -0.167480 -0.385254  9.742188  1658352657738     None   
1       187379909791333 -0.174683 -0.390137  9.734375  1658352657738     None   
2       187379919691333 -0.160400 -0.308838  9.742188  1658352657917     None   
3       187379929591333 -0.143555 -0.284668  9.734375  1658352657917     None   
4       187379939491333 -0.165161 -0.337402  9.765625  1658352657918     None   
...                 ...       ...       ...       ...            ...      ...   
179168  258404779038026 -1.792969 -0.368652  9.632812  1658423682449     None   
179169  258404979038026 -1.797852 -0.380615  9.609375  1658423682651     None   
179170  258405179038026 -1.816406 -0.375732  9.61718